In [ ]:
reply = ChatbotResponse.getResponse("how far is kelowna from vancouver?")
print(reply)

Success
The distance between kelowna and vancouver is 270.78457831176075km.


In [ ]:
wa = WitAPI()
lc = Location()
rf = ResponseFormat()
tc = TimeChatbot()
gi = GeoInfo()

import json
"""from witapi import WitAPI as wa
from location import Location as lc
from responseformat import ResponseFormat as rf
from timechatbot import TimeChatbot as tc
from geoInfo import GeoInfo as gi"""


witIntents = {
	'get_distance' : lc.distanceByLatLong,
	'get_time_difference' : tc.getTimeDifference,	
	'get_time_zone' : tc.getTimezone,
	'point_of_interest' : gi.get_point_of_interest,
	'forecast' : gi.get_weather,
	'wit$get_temperature' : gi.get_temperature,
	'wit$get_time' : tc.getLocalTime
}

witIntentResponseFormats = {
	'get_distance' : rf.getDistanceFormat,
	'get_time_difference' : rf.getTimeDifferenceFormat,
	'get_time_zone' : rf.getTimezoneFormat,
	'point_of_interest' : rf.getPointOfInterestFormat,
	'forecast' : rf.getWeatherFormat,
	'wit$get_temperature' : rf.getTemperatureFormat,
	'wit$get_time' : rf.getLocalTimeFormat
}

class ChatbotResponse(object):

  def getResponse(question):
    jsonData = wa.sendRequest(question)
    #print(jsonData)

    response = ""

    intents = jsonData['intents']
    entities = jsonData['entities']
    traits = jsonData['traits']
    #print('---------', entities['wit$location:location'][0]['value'])
    if len(intents) == 0:
      response = "I'm sorry, I didn't quite understand that. Try asking for help to see the scope of my functionality, or try asking another question."
    else:
      for intent in intents:
        #print(intent)
        intentName = intent['name']
        #print('---------', entities)
        #print('+++++++++++', witIntents[intentName](jsonData))
        response += witIntentResponseFormats[intentName](witIntents[intentName](entities), entities)

    return response


print(ChatbotResponse.getResponse("What is the weather forecast in Kelowna?"))

Success
The weather in Kelowna is clear sky.


In [ ]:
from datetime import datetime
from datetime import timedelta
import json 
import requests

URL = "https://api.wit.ai/message"
API_KEY = "6LLX376UX4FWFSX5KVNLSRPCXSECLGXE"
headers  = {
	"Authorization" : "Bearer " + API_KEY
}

class WitAPI(object):

	def sendRequest(self, question):
		date = datetime.now()
		date = date.strftime("%Y%m%d")
		resp = requests.get(URL + "?v=" + date + "&q=" + question, headers = headers)

		if resp.status_code != 200:
		    print('error: ' + str(resp.status_code))
		    return None
		else:
		    print('Success')
		    return resp.json()

wa.sendRequest('What is the weather forecast in Kelowna?')
wa.sendRequest('What is the weather forecast in Kelowna and Toronto?')

Success
Success


{'entities': {'wit$location:location': [{'body': 'Kelowna',
    'confidence': 0.9945,
    'end': 39,
    'entities': [],
    'id': '184461976774247',
    'name': 'wit$location',
    'role': 'location',
    'start': 32,
    'suggested': True,
    'type': 'value',
    'value': 'Kelowna'}]},
 'intents': [],
 'text': 'What is the weather forecast in Kelowna and Toronto?',
 'traits': {}}

In [ ]:
"""from lib.timezonefinder import TimezoneFinder
from datetime import datetime
from location import Location as lc
from lib import pytz"""

#! pip install timezonefinder[numba]
from timezonefinder import TimezoneFinder
from datetime import datetime
import pytz


class TimeChatbot(object):

	def getTimezone(self, entities, place = None):
		if place is None:
			timezones = []
			entities = entities['wit$location:location']
			for location in entities:
				coords = lc.getLocation(location['value'])
				latitude = coords[0]
				longitude = coords[1]
				tf = TimezoneFinder()
				timezones.append(tf.timezone_at(lng=longitude, lat=latitude))
			return timezones
		else:
			coords = lc.getLocation(place)
			latitude = coords[0]
			longitude = coords[1]
			tf = TimezoneFinder()
			return tf.timezone_at(lng=longitude, lat=latitude)

	def getLocalTime(self, entities, place = None):
		locationEntities = entities['wit$location:location']
		if place is None:
			dateTime = [] 
			if len(locationEntities) == 0:
				dateTime.append(datetime.now())
			else:
				timezoneList = self.getTimezone(entities)
				for timezone in timezoneList:
					tz = pytz.timezone(timezone)
					dateTime.append(datetime.now(tz))
			return dateTime
		else:
			timezone = self.getTimezone(entities, place)
			tz = pytz.timezone(timezone)
			return datetime.now(tz)

	def getTimeDifference(self, entities):
		times = self.getLocalTime(entities)
		if len(times) == 1:
			times.append(self.getLocalTime(entities, "here"))

		return times[0] - times[1]

# testing for this class:
tmp = TimeChatbot()
#tmp.getTimezone(entities['entities'])
tmp.getTimezone(entities['entities'], 'New York')
tmp.getLocalTime(entities['entities'], 'New York')
tmp.getTimeDifference(entities['entities'])

datetime.timedelta(days=-1, seconds=86399, microseconds=834576)

In [ ]:


class ResponseFormat(object):
	def getDistanceFormat(self, distance, entities):
		entities = entities['wit$location:location']
		place1 = entities[0]['value']
		if len(entities) == 1:
			return "The distance to " + place1 + " is " + str(distance) + "km."
		else:
			place2 = entities[1]['value']
			return "The distance between " + place1 + " and " + place2 + " is " + str(distance) + "km."

	def getTimezoneFormat(self, timezone, entities):
		entities = entities['wit$location:location']
		place = entities[0]['value']
		return "The time zone at " + place + " is " + timezone + "."

	def getLocalTimeFormat(self, time, entities):
		entities = entities['wit$location:location']
		response = ""
		i = 0
		for location in [entities[0]['value']]: # there may be some problems
			response += "The time in " + location + " is " + str(time[i]) + ".\n"
			i = i + 1
		return response

	def getTimeDifferenceFormat(self, time, entities):
		entities = entities['wit$location:location']
		if len(entities) == 1:
			return "The difference in time from here to " + entities[0]['value'] + " is " + str(time) + "."
		else:
			return "The difference in time between " + entities[0]['value'] + " and " + entities[1]['value'] + " is " + str(time) + "."

	def getTemperatureFormat(self, temps, entities):
		entities = entities['wit$location:location']
		response = ""
		i = 0
		for location in entities:
			response += "The temperature in " + location['value'] + " is " + str(temps[i]['temp']) + " degrees celcius.\n"
			i = i + 1
		return response

	def getWeatherFormat(self, weather, entities):
		entities = entities['wit$location:location']
		return "The weather in " + entities[0]['value'] + " is " + str(weather) + "."

	def getPointOfInterestFormat(self, POI, entities):
		entities = entities['wit$location:location']
		response = "The POIs in " + entities[0]['value'] + " are "
		for poi in POI:
			stringsplitpoint = poi.find(', ') + 2
			response += poi[stringsplitpoint:poi.find(',', stringsplitpoint+1)] + ", "
		response = response[:-2]
		response += "."
		return response

# test for this class:
tmp = ResponseFormat()
tmp.getDistanceFormat(1000, entities['entities'])
tmp.getTimezoneFormat('Toronto timezone', entities['entities'])
time01 = ['06:06:00', '12:00:00','06:06:00', '12:00:00']
tmp.getLocalTimeFormat(time01, entities['entities'])
tmp.getTimeDifferenceFormat('1 day', entities['entities'])
temp01 = [{'feels_like': -7.35,
  'temp': -3.52,
  'temp_kf': None,
  'temp_max': -2.78,
  'temp_min': -4.0}]
#print(temp01)
tmp.getTemperatureFormat(temp01, entities['entities'])
tmp.getWeatherFormat('clear sky', entities['entities'])
poi01 = ['distance: 113, Kelowna Cruises, 238 Queensway, Kelowna, BC V1Y, Canada',
 'distance: 116, Hydrofly Kelowna, 230 Queensway, Kelowna, BC V1Y, Canada',
 'distance: 124, Kelownacruises - Lake, 239 Queensway, Kelowna, BC V1Y, Canada',
 'distance: 140, Okanagan Heritage Museum, 470 Queensway, Kelowna, BC V1Y, Canada',
 'distance: 140, Okanagan Heritage Museum, 470 Queensway, Kelowna, BC V1Y, Canada']
#print(poi01)
tmp.getPointOfInterestFormat(poi01, entities['entities'])


'The POIs in Kelowna are Kelowna Cruises, Hydrofly Kelowna, Kelownacruises - Lake, Okanagan Heritage Museum, Okanagan Heritage Museum.'

In [ ]:
"""from lib.geopy.geocoders import Nominatim
from lib.geopy import distance
from lib import ipregistry"""
#! pip install ipregistry
from ipregistry import IpregistryClient
from geopy.geocoders import Nominatim
import geopy.distance

class Location(object):

  def __init__(self, place = None):
    self.lat = 0
    self.long = 0
    if place is None or place == 'here' or place == 'me':
      #client = ipregistry.IpregistryClient("fs9pbuwmnx5r2g", cache=ipregistry.DefaultCache(maxsize=2048, ttl=600))  
      client = IpregistryClient("fs9pbuwmnx5r2g")
      ipInfo = client.lookup()
      #print(ipInfo)
      self.lat = ipInfo.location.get("latitude")
      self.long = ipInfo.location.get("longitude")
    else:
      geolocator = Nominatim(user_agent="chatbot")
      location = geolocator.geocode(place)
      self.lat = location.latitude
      self.long = location.longitude

  def getLocation(self, place = None):
    if place is None or place == 'here' or place == 'me':
      #client = ipregistry.IpregistryClient("fs9pbuwmnx5r2g", cache=ipregistry.DefaultCache(maxsize=2048, ttl=600))  
      """client = IpregistryClient("fs9pbuwmnx5r2g")
      ipInfo = client.lookup()
      return (ipInfo.location.get("latitude"), ipInfo.location.get("longitude"))"""
      return (self.lat, self.long)
    else:
      geolocator = Nominatim(user_agent="chatbot")
      location = geolocator.geocode(place)
      return (location.latitude, location.longitude)

  def distanceByLatLong(self, entities):
    places = entities['wit$location:location']
    #print('--------', len(places))
    place1Coords = self.getLocation(places[0]['value']) # Kelowna
    if len(places) == 1: # len(places) always 1
      place2Coords = self.getLocation()
    else:
      place2Coords = self.getLocation(places[1]['value'])

    return geopy.distance.distance(place1Coords, place2Coords).km

# test for this class:
tmp = Location('New York')
tmp.getLocation() # to Kelowna
#print(entities)
tmp.distanceByLatLong(entities['entities'])

3648.980428105776

In [ ]:
#you should install the package firstly
#!pip install pyowm 
"""from lib.pyowm import OWM
from lib.pyowm.utils import config
from lib.pyowm.utils import timestamps"""

#!pip install pyowm 
from pyowm import OWM
from pyowm.utils import config
from pyowm.utils import timestamps
import json 
import requests

import json 
import requests
#from location import Location as lc

# you should go to this URL: https://home.openweathermap.org/api_keys
# to create your own key if you want to use this API
# more info about this API: https://openweathermap.org/api/one-call-api
api_key = "817d0ac612d439a9fe94c38f889fab28"
owm = OWM(api_key)
mgr = owm.weather_manager()

class GeoInfo(object):


  def __init__(self, lat=43.6532, long=-79.3832):
    self.lat = lat
    self.long = long
    self.observation = mgr.weather_at_coords(self.lat, self.long)
    self.w = self.observation.weather
  
  def get_temperature(self, entities, lat=None, long=None, wtype='celsius'):
    locationEntities = entities['wit$location:location']
    temps = []
    if len(locationEntities) == 0:
      coords = lc.getLocation()
      lat = coords[0]
      long = coords[1]
      temps.append(mgr.weather_at_coords(lat, long).weather.temperature(wtype))
    else:
      for location in locationEntities:
        #print('==========', location['value'])
        coords = lc.getLocation(location['value'])
        lat = coords[0]
        long = coords[1]
        temps.append(mgr.weather_at_coords(lat, long).weather.temperature(wtype))
    #observation = mgr.weather_at_coords(lat, long)
    #w = observation.weather
    # print(w.temperature('celsius'))
    # w.to_dict()
    return temps
  '''
  def get_temperature(self, name='Toronto'):
    observation = mgr.weather_at_place(name)
    w = observation.weather
    print(w.temperature('celsius'))
    return w.to_dict()
  '''

  def get_weather(self, entities, lat = 0, long = 0):
    #print('===========')
    locationEntities = entities['wit$location:location']
    if len(locationEntities) == 0:
      coords = lc.getLocation()
      lat = coords[0]
      long = coords[1]
    else:
      coords = lc.getLocation(locationEntities[0]['value'])
      lat = coords[0]
      long = coords[1]
    return mgr.weather_at_coords(lat, long).weather.detailed_status
  
  # limit, how many POIs you want to retrieve
  def get_point_of_interest(self, entities, limit=5, latitude = 0, longitude = 0):
    URL = "https://discover.search.hereapi.com/v1/discover"
    locationEntities = entities['wit$location:location']
    if len(locationEntities) == 0:
      coords = lc.getLocation()
      latitude = coords[0]
      longitude = coords[1]
    else:
      coords = lc.getLocation(locationEntities[0]['value'])
      latitude = coords[0]
      longitude = coords[1]
    # get your key from developer.here.com
    poi_key = 'IIlHrKDojjhW90OB3KCEp6Xu0l8FX13NAdgFxMx4HbM' 
    query = 'tourist attractions'
    #print(latitude, longitude)
    PARAMS = {
          'apikey':poi_key,
          'q':query,
          'limit': limit,
          'at':'{},{}'.format(latitude,longitude)
          } 
    # sending request and getting the response as JSON
    r = requests.get(url = URL, params = PARAMS) 
    data = r.json()
    items = data['items']
    attractions = []
    # parse retrieved attractions data
    for it in items:
      att = 'distance: ' + str(it['distance']) + ', ' + it['address']['label']
      attractions.append(att)
    return attractions

# test for this class:
entities = {'entities': {'wit$location:location': [{'body': 'Kelowna',
    'confidence': 0.9962,
    'end': 39,
    'entities': [],
    'id': '184461976774247',
    'name': 'wit$location',
    'role': 'location',
    'start': 32,
    'suggested': True,
    'type': 'value',
    'value': 'Kelowna'}]},
 'intents': [{'confidence': 0.9459,
   'id': '1062561500902258',
   'name': 'forecast'}],
 'text': 'What is the weather forecast in Kelowna?',
 'traits': {}}

#print(entities)
tmp = GeoInfo()
tmp.get_temperature(entities['entities'])
tmp.get_weather(entities['entities'])
tmp.get_point_of_interest(entities['entities'])

['distance: 113, Kelowna Cruises, 238 Queensway, Kelowna, BC V1Y, Canada',
 'distance: 116, Hydrofly Kelowna, 230 Queensway, Kelowna, BC V1Y, Canada',
 'distance: 124, Kelownacruises - Lake, 239 Queensway, Kelowna, BC V1Y, Canada',
 'distance: 140, Okanagan Heritage Museum, 470 Queensway, Kelowna, BC V1Y, Canada',
 'distance: 140, Okanagan Heritage Museum, 470 Queensway, Kelowna, BC V1Y, Canada']